# **DCDC Ejercicio T3.4 ARIMA, SARIMA, SARIMAX<a id="top"></a>**

<i><small>**Alumno:** Alejandro Pequeño<br>Última actualización: 3/12/2023</small></i></div>

---

**Índice:**

- [Imports](#-Imports)


- [1. Carga y visualización de datos](##-1.-Carga-y-visualización-de-datos)

<lr>

- [2. Análisis del DataFrame](##-2.-Análisis-del-DataFrame)
  - [2.1 Descomposición de la serie temporal](###-2.1-Descomposición-de-la-serie-temporal)
  - [2.2 Prueba de Dickey-Fuller](###-2.2-Prueba-de-Dickey-Fuller)
  - [2.3 Análisis de la autocorrelación](###-3.2-Análisis-de-la-autocorrelación)

<lr>

- [3. Métodos de Predicción](#-3.-Métodos-de-Predicción)
  - [3.0 AR, MA, ARMA](###-3.1-AR,-MA,-ARMA)
  - [3.1 ARIMA](###-3.1-ARIMA)
  - [3.2 SARIMA](###-3.2-SARIMA)
  - [3.3 SARIMAX](###-3.3-SARIMAX)

<lr>


## Imports

In [1]:
# Imports
# =====================================================================
!pip install statsmodels
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import scipy.stats
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.filters.hp_filter import hpfilter
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from statsmodels.tsa.statespace.sarimax import SARIMAX

Defaulting to user installation because normal site-packages is not writeable
